## 気楽に始める scikit-learn 講座

### 目次
1. scilit-learnとは？
1. 世界一簡単なモデル作成
1. ちょっと深掘り
1. モデル評価
1. モデル保存と再利用
1. 実際に分析してみる
1. その他便利機能
1. まとめ

In [1]:
import warnings
# warningをオフにする (ほんとはだめ)
warnings.simplefilter('ignore')
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

### 1. scikit-learnとは？
* `scikit-learn` は現在最も普及している機械学習ライブラリです。  
* 数行でモデルが作成できる手軽さから、機械学習エンジニア以外でも容易に使うことができるのが素敵です。
* 前処理や性能評価など、モデル作成以外の機能も提供されているため、とりあえず困ったら `sklearn` を調べることが多いです。 (javascriptで言うところの、 `lodash` 的な…
* 他の機械学習ライブラリも `sklearnライク` なインターフェースを提供していることも多く、デファクトスタンダード的な存在になっています。
* **とりあえずこれを覚えておけ!** ライブラリです。

### 2. 世界一簡単なモデル作成

解説は後回しで `sklearn` の手軽さを体験します。  
何はともあれデータのロードです (ちなみにガン診断のデータです

In [2]:
data = load_breast_cancer()

データが揃ったので、モデルを作成してみます。  
今回は患者のデータから、その患者がガンかどうかを判定します。

In [3]:
# モデルの作成
model = LogisticRegression()
# 学習の開始
model.fit(data.data, data.target)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

これだけで学習モデルが完成しました。  
モデルが出来たので、ちょっと予測してみます

In [4]:
print('predicted value: ', model.predict(data.data[3].reshape(1, -1)))
print('actual value: ', data.target[3])

predicted value:  [1]
actual value:  0


ちゃんと外れてますね…笑

### 3. ちょっと深掘り

#### インターフェースの利用
`sklaern` にはたくさんの機械学習アルゴリズムが用意されていますが、使い方は統一されています。

1. `hogemodel()` モデルで外枠を作成
2. `fit()` で学習開始
3. `predict()` で予測


たった3ステップです。  そこで他のモデルも試してみます。

In [5]:
models = [LogisticRegression(), SVC(), RandomForestClassifier()]
trained_models = []

# 3つ分のモデルを作成
for model in models:
    model.fit(data.data, data.target)
    trained_models.append(model)

In [6]:
# RandomForestで検証
print('predicted value: ', trained_models[2].predict(data.data[3].reshape(1, -1)))
print('actual value: ', data.target[3])

predicted value:  [0]
actual value:  0


#### `hogemodel` について
* `sklearn` には多種多様なモデルが用意されていますが、各モデルの引数には、それぞれに合わせたモデルのオプションを指定します。  
* これを外から設定するパラメータにちなんで **ハイパーパラメータ** と言います。  
* ただし、sklearnには **デフォルトのハイパーパラメータでもうまくいくこと** が設計思想として盛り込まれています。

In [7]:
# ハイパーパラメータ C を変えてみる
model1 = LogisticRegression(C=10, random_state=42)
model2 = LogisticRegression(C=0.1, random_state=42)
# なので推定結果が異なる
model1.fit(data.data, data.target)
model2.fit(data.data, data.target)
print('predicted value: ', model1.predict(data.data[3].reshape(1, -1)))
print('predicted value: ', model2.predict(data.data[3].reshape(1, -1)))

predicted value:  [0]
predicted value:  [1]


#### `fit`メソッドについて
* モデルの学習は`fit`メソッドで行いますが、第一引数は学習データ, 第二引数は教師データです。
* `fit` メソッド自体は単純ですが、これを行った後のmodelインスタンスは、学習済みモデルとなり、色々な情報が付加されています


In [8]:
model = LogisticRegression(C=0.1, random_state=42)
model.fit(data.data, data.target)
# 推定されたパラメータを表示
print(model.coef_)

[[ 0.66961014  0.08701507  0.3194159  -0.01172696 -0.02583376 -0.11308581
  -0.16161837 -0.07059695 -0.03609787 -0.00644475  0.02286078  0.28965259
   0.05778792 -0.07476975 -0.00254652 -0.02051683 -0.03054616 -0.00904964
  -0.0083585  -0.00165526  0.66693723 -0.23022732 -0.21636029 -0.01551808
  -0.04769816 -0.34625588 -0.43524746 -0.13641814 -0.1148247  -0.03257108]]


#### `predict`メソッドについて
* 学習済みのモデルに対して、予測を行う際には、`predict`メソッドを用います。
* 使用したモデルによって、0or1を返すもの、10とか100返すもの様々です。
* 0or1を返すものは、確率値を返す`predict_proba`メソッドも使えます。

In [9]:
print('predicted value: ', model.predict(data.data[3].reshape(1, -1)))
print('predicted proba: ', model.predict_proba(data.data[3].reshape(1, -1)))

predicted value:  [1]
predicted proba:  [[0.24566492 0.75433508]]


### 3.モデルの評価

* 学習モデルが本当によかったかどうかは、ちゃんと評価する必要があります。  
* モデル評価用のメソッドももちろん用意されていますが、ここでは過学習について考えてみます。
* 学習用と評価用のデータに分けて、未知のデータに対しても上手く予測できるモデルが良いモデルとなります。

#### `train_test_split`メソッド
* このメソッドは学習データとテストデータに分けるメソッドです。
* ポイントはテストデータの中身を見ないことです。カンニングになります。

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    data.data, data.target, test_size=0.2, random_state=42)
print('shape X_train:', X_train.shape)
print('shape X_test: ', X_test.shape)

shape X_train: (455, 30)
shape X_test:  (114, 30)


* 学習データとテストデータを作成したので、学習してみます。

In [11]:
model = LogisticRegression(C=10000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(C=10000, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=42, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

#### `score`メソッド
* scoreメソッドは、二値分類ではタスクでは、正答率(accuracy)と呼ばれる指標を計算します。
* もちろん正答率以外の指標を計算するメソッドもたくさん用意されていて、状況に応じて使い分ける必要があります。

In [12]:
print(model.score(X_train, y_train))
print(model.score(X_test, y_test))

0.9802197802197802
0.9649122807017544


* スコアみると学習データよりテストデータのスコアが低くなっています。
* このように学習データに過剰に適合してしまっている状態を **過学習** と呼びます。
* これでは未知のデータに対して上手く予測出来ないので、良いモデルとは言えません。
* 目指すべきは正答率が高く、かつ学習データとテストデータの正答率が同じくらいのモデルです。

### 5. モデルの保存と再利用

* 実際の運用ではリアルタイムの学習を行うことは、技術的、費用的コストが高いので、学習したモデルを保存しておき、再利用することが多いです。
* 学習済みモデルをロード、`predict`メソッドの結果を返すのを関数化すると、それだけで立派な機械学習APIの出来あがりです。デバイスのEdgeに入れるのもありです。

`joblib`ライブラリ
* 学習済みモデルを外部ファイルに保存します。
* ファイルは単なるバイナリなので、拡張子に指定はないですが、`pkl`が使われることが多いです。(漬物です
* `dump`メソッドでモデルを保存します。

In [13]:
from sklearn.externals import joblib
joblib.dump(model, 'Logistic.pkl')

/Users/soichiro.fujioka/ds-venv/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


['Logistic.pkl']

* 保存が完了したので、一旦モデルを削除し、ロードしてみます。

In [14]:
# モデルを削除
del model
# モデルを読み込み
model = joblib.load('Logistic.pkl')
print('predicted value', model.score(X_test, y_test))

predicted value 0.9649122807017544


### 6. 実際に分析してみる

* ここまでの解説を踏まえて実際に分析してみたいと思います。
* 実はここからがハンズオンです。
* 今回はタイタニック号沈没事件の生存データを使ってみます。

#### データのロードと(簡単な前処理)

In [15]:
import pandas as pd #データフレームライブラリ
titanic = pd.read_csv('./data/titanic.csv')
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


* `Survived` が生存したどうかのデータなので、その他のデータを使って予測してみます。

In [16]:
# 欠損値を確認
titanic.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [17]:
# 学習データを教師データを分ける + 処理がめんどくさそうなデータを除く
y_data = titanic["Survived"]
X_data = titanic.drop(["PassengerId", "Survived", "Name", "Ticket", "Cabin"], axis=1)

In [18]:
# 文字列データはそのままでは扱えないので、数値データに変換
X_data["Sex"] = X_data["Sex"].replace("male", 1)
X_data["Sex"] = X_data["Sex"].replace("female", 0)

In [19]:
# 欠損値をカテゴリとして扱う
X_data["Embarked"] = X_data["Embarked"].fillna('Na')
# ダミー変数化
dummies = pd.get_dummies(X_data["Embarked"])
X_data['Q'] = dummies['Q']
X_data['S'] = dummies['S']
X_data['C'] = dummies['C']
X_data.drop(["Embarked"], axis=1, inplace=True)

In [20]:
import numpy as np
# Ageの欠損値を平均値で補完する
X_data["Age"] = X_data["Age"].fillna(np.mean(X_data["Age"]))

* ひとまずこれで最低限分析できるようになりました

### 学習データとテストデータに分ける

In [21]:
X_train, X_test, y_train, y_test = train_test_split(
    X_data, y_data, test_size=0.2, random_state=42)

* いくつかモデルを用意したので、モデルとパラメータを変えてみて、結果を違いを見てみましょう！
* `command + /` で行のコメントアウトができます。

In [22]:
model = LogisticRegression(C=10, random_state=42) # C >= 0 で変更可能
# model = SVC(C=1000, random_state=42) # C > 0 で変更可能
# model = RandomForestClassifier(n_estimators=10 ,random_state=42) # n_estimators > 0 で変更可能
# model = MLPClassifier(hidden_layer_sizes=(3, 10), random_state=42) # hidden_layer_sizesはタプルで共に1以上で変更可能
model.fit(X_train, y_train) 
# 解析結果を表示
print("train score: ", model.score(X_train, y_train))
print("test score:  ", model.score(X_test, y_test))

train score:  0.8019662921348315
test score:   0.8100558659217877


### 7. その他便利機能

* ここまでモデルの構築をメインに`sklearn`の使い方を説明しましたが、その他にも処理を楽にする機能が用意されています。
* ここではほんの一部を紹介します。

#### データの前処理

In [23]:
data = [[0, 0], [0, 0], [1, 1], [1, 1]]
scaler = StandardScaler()
scaler.fit(data)
scaler.transform(data)

array([[-1., -1.],
       [-1., -1.],
       [ 1.,  1.],
       [ 1.,  1.]])

`StandardScaler`はデータの分散を1に平均を0にする操作です。数値計算の安定化の為によく使われます。

#### 文字列データを数値に変換

In [24]:
data = [['Male'], ['Female'], ['Female']]
oneHotEncoder = OneHotEncoder()
oneHotEncoder.fit(data)
oneHotEncoder.transform(data).toarray()

array([[0., 1.],
       [1., 0.],
       [1., 0.]])

#### パイプライン
* モデルにデータを食わせるまでのフローは固定されていることが多い為、パイプラインとして処理を構築するとスッキリとします。(一つのAPIの処理の流れが出来上がるイメージです。

In [25]:
step = [
    ('scaler', StandardScaler()),
    ('model', LogisticRegression())
]
pipeline = Pipeline(step)
pipeline.fit(X_train, y_train)
print('predicted value', pipeline.predict(X_train[0:3]))

predicted value [0 0 0]


In [26]:
print("train score: ", pipeline.score(X_train, y_train))
print("test score:  ", pipeline.score(X_test, y_test))

train score:  0.8019662921348315
test score:   0.8100558659217877


* ここでは割愛しますが、先ほど手入力で行っていた、パラメータのチューニングや、より高度な精度評価(KFold CV)も行うことができます。

### 8.まとめ

#### ここがGood
* `sklearn`は簡単な記述でモデルを作成することができます。
* 機械学習モデルの構築だけでなく、データの前処理やハイパーパラメータのチューニングを行うこともできます。
* 基本的な機械学習タスクからニューラルネットまで構築できるので、とりあえず **POC**的作成しておくという使い方もできます
* 一番使われているフレームワークなので、日本語資料もたくさんあります。

#### ここがWeak
* ディープニューラルネットやその他の高度な手法は別ライブラリに頼る必要がある(Tensorflow, Pytorchとか)
* GPUが使えない